In [9]:
import tensorcircuit as tc
import numpy as np
import matplotlib.pyplot as plt
import time

tc.set_backend("numpy")
tc.set_dtype("complex128")


('complex128', 'float64')

In [10]:

def gen_H_GAA_nambu(L, t, lbd, a, b, phi):
    H = np.zeros((2 * L, 2 * L), dtype=np.complex128)

    for i in range(L - 1):
        H[i, i + 1] = t / 2
        H[i + 1, i] = t / 2
        H[i + L, i + 1 + L] = -t / 2
        H[i + 1 + L, i + L] = -t / 2

    for i in range(L):
        H[i, i] = 2 * lbd * np.cos(2*np.pi*i*b+phi) / (1-a*np.cos(2*np.pi*i*b+phi)) / 2
        H[i+L, i+L] = -H[i, i]

    return H


In [11]:

def gen_psi_0(state_name, L, f):
    if state_name == "random_state":
        filled_indices = np.random.choice(np.arange(L), size=L//f, replace=False)
        # .choice 是随机选择给定序列中的元素，组成 size 大小的数组，这里最后一个参数为 False 表示选取不重复
        alpha = tc.FGSSimulator.init_alpha(filled_indices, L)
    elif state_name == "neel_state":
        filled_indices = np.arange(0, L, f)
        alpha = tc.FGSSimulator.init_alpha(filled_indices, L)
    elif state_name == "bipartite_state":
        filled_indices = np.arange(0 , L//f)
        alpha = tc.FGSSimulator.init_alpha(filled_indices, L)
    else:
        print("state name is wrong")
        exit(1)
    return alpha


In [12]:
help(tc.FGSSimulator)

Help on class FGSSimulator in module tensorcircuit.fgs:

class FGSSimulator(builtins.object)
 |  FGSSimulator(L: int, filled: Optional[List[int]] = None, alpha: Optional[Any] = None, hc: Optional[Any] = None, cmatrix: Optional[Any] = None)
 |
 |  Fermion Gaussian State (FGS) simulator for efficient simulation of non-interacting fermionic systems.
 |
 |  This simulator implements methods for:
 |  1. State initialization and evolution
 |  2. Correlation matrix calculations
 |  3. Entanglement measures
 |  4. Out-of-time-ordered correlators (OTOC)
 |  5. Post-selection and measurements
 |
 |  Main references:
 |  - Gaussian formalism: https://arxiv.org/pdf/2306.16595.pdf
 |  - Quantum circuits: https://arxiv.org/abs/2209.06945
 |  - Theory background: https://scipost.org/SciPostPhysLectNotes.54/pdf
 |
 |  Conventions:
 |  - Hamiltonian form: (c^dagger, c)H(c, c^\dagger)
 |  - Correlation form: <(c, c^\dagger)(c^\dagger, c)>
 |  - Bogoliubov transformation: c' = \alpha^\dagger (c, c^\dagge

In [13]:

def cal_S_and_N_of_t(state_name, L, t, lbd_array, a, b, phi, steps, dt, f):
    S_array = np.zeros((len(lbd_array), steps), dtype=np.complex128)
    N_array = np.zeros((len(lbd_array), steps, L), dtype=np.complex128)
    sub_system = list(range(L // 2))

    if state_name == "random_state":
        random_sample = 20
        alpha_0_array = [gen_psi_0(state_name, L, f) for _ in range(random_sample)]
        for r, alpha_0 in enumerate(alpha_0_array):
            for i, lbd in enumerate(lbd_array):
                print(f"r = {r+1} / {random_sample}, ldb = {lbd:.2f} ({i + 1} / {len(lbd_array)})")
                H_GAA_nambu = gen_H_GAA_nambu(L, t, lbd, a, b, phi)
                system = tc.FGSSimulator(L, alpha=alpha_0)

                for j in range(steps):
                    S_array[i, j] += system.entropy(sub_system)
                    N_array[i, j, :] += [system.expectation_2body(i + L, i) for i in range(L)]
                    system.evol_ghamiltonian(2 * H_GAA_nambu * dt)
        S_array /= random_sample

    else:
        alpha_0 = gen_psi_0(state_name, L, f)
        for i, lbd in enumerate(lbd_array):
            print(f"ldb = {lbd:.2f} ({i + 1} / {len(lbd_array)})")
            H_GAA_nambu = gen_H_GAA_nambu(L, t, lbd, a, b, phi)
            system = tc.FGSSimulator(L, alpha=alpha_0)

            for j in range(steps):
                S_array[i, j] = system.entropy(sub_system)
                N_array[i, j, :] = [system.expectation_2body(i + L, i) for i in range(L)]
                system.evol_ghamiltonian(2 * H_GAA_nambu * dt)

    file_name = f"2 - S_{state_name}_f_{f}_L_{L}_a_{a:.1f}_lbd_{lbd_array[0]}_{lbd_array[-1]}_steps_{steps}_dt_{dt}"
    np.savez("data//" + file_name + ".npz", S_array=S_array)
    file_name = f"2 - N_{state_name}_f_{f}_L_{L}_a_{a:.1f}_lbd_{lbd_array[0]}_{lbd_array[-1]}_steps_{steps}_dt_{dt}"
    np.savez("data//" + file_name + ".npz", N_array=N_array)


In [14]:

def vis_S_of_t(L, lbd_array, steps, dt, a, f):
    file_name = f"2 - S_{state_name}_f_{f}_L_{L}_a_{a:.1f}_lbd_{lbd_array[0]}_{lbd_array[-1]}_steps_{steps}_dt_{dt}"
    data = np.load("data//" + file_name + ".npz")
    S_array = data["S_array"]

    plt.figure(figsize=(10, 6))
    for i, lbd in enumerate(lbd_array):
        plt.plot(np.arange(1, steps+1e-3), S_array[i, :], marker=".", linewidth=2, label=r"$\lambda=%.2f$" % (lbd))
    
    plt.title(state_name)
    plt.legend()
    plt.xlabel(rf"$steps/{dt}$")
    plt.xlim(1, steps)
    plt.ylabel(r"$S$")
    plt.tight_layout()
    plt.savefig("fig//" + file_name + ".png", dpi=300, bbox_inches="tight")


In [15]:

def vis_N_of_t(L, lbd_array, steps, dt, a, f):
    file_name = f"2 - N_{state_name}_f_{f}_L_{L}_a_{a:.1f}_lbd_{lbd_array[0]}_{lbd_array[-1]}_steps_{steps}_dt_{dt}"
    data = np.load("data//" + file_name + ".npz")
    N_array = data["N_array"]
    n = 0
    N_array = N_array[0, :, :]

    plt.figure(figsize=(10, 6))
    plt.pcolormesh(np.real(N_array.T), cmap="viridis")
    
    plt.colorbar()
    plt.title(state_name + rf' $\lambda={lbd_array[n]:.2f}$')
    plt.xlabel(rf"$steps/{dt}$")
    plt.ylabel(r"$N$")
    plt.tight_layout()
    plt.savefig("fig//" + file_name + f"_lbd={lbd_array[n]:.2f}.png", dpi=300, bbox_inches="tight")


In [16]:


if __name__ == "__main__":
    np.random.seed(100)
    state_name = "bipartite_state"
    f = 4
    L = 200
    t = 1
    lbd_array = np.concatenate((np.arange(0, 0.5, 0.25), np.arange(0.5, 1.5, 0.1), np.arange(1.5, 2.0+1e-3, 0.25)))
    a = 0.3
    b = 2 / (np.sqrt(5) - 1)
    phi = 0
    steps = 1000
    dt = 100

    start_time = time.time()
    # cal_S_and_N_of_t(state_name, L, t, lbd_array, a, b, phi, steps, dt, f)
    vis_S_of_t(L, lbd_array, steps, dt, a, f)
    vis_N_of_t(L, lbd_array, steps, dt, a, f)
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.1f} s")


FileNotFoundError: [Errno 2] No such file or directory: 'data//2 - S_bipartite_state_f_4_L_200_a_0.3_lbd_0.0_2.0_steps_1000_dt_100.npz'